
# MMF1921/MIE377 - Backtesting Template

The purpose of this program is to provide a template with which to develop Project 2. The project requires you to test different models  (and/or different model combinations) to create an asset management algorithm.

This template will be used by the instructor and TA to assess your trading algorithm using different datasets.

# PLEASE DO NOT MODIFY THIS TEMPLATE (for Project submission purposes)

# 1. Read input files

In [1]:
import time
import math
from scipy.stats import gmean
import matplotlib.pyplot as plt
from services.project_function import *
import pandas as pd

adjClose = pd.read_csv("MIE377_AssetPrices_3.csv", index_col=0)
factorRet = pd.read_csv("MIE377_FactorReturns_3.csv", index_col=0)

In [2]:
adjClose.index = pd.to_datetime(adjClose.index)
factorRet.index = pd.to_datetime(factorRet.index)

In [3]:
# Initial budget to invest ($100,000)
initialVal = 100000

# Length of investment period (in months)
investPeriod = 6

# divide the factor returns by  100
factorRet = factorRet/100

#rf and factor returns
riskFree = factorRet['RF']
factorRet = factorRet.loc[:,factorRet.columns != 'RF'];


In [4]:
#Identify the tickers and the dates
tickers = adjClose.columns
dates   = factorRet.index

In [5]:
# Calculate the stocks monthly excess returns
# pct change and drop the first null observation
returns = adjClose.pct_change(1).iloc[1:, :]
returns = returns  - np.diag(riskFree.values) @ np.ones_like(returns.values)
# Align the price table to the asset and factor returns tables by discarding the first observation.
adjClose = adjClose.iloc[1:,:]

In [6]:
assert adjClose.index[0] == returns.index[0]
assert adjClose.index[0] == factorRet.index[0]

In [7]:
adjClose = pd.read_csv("MIE377_AssetPrices_2.csv", index_col=0)
factorRet = pd.read_csv("MIE377_FactorReturns_2.csv", index_col=0)

adjClose.index = pd.to_datetime(adjClose.index)
factorRet.index = pd.to_datetime(factorRet.index)

# Initial budget to invest ($100,000)
initialVal = 100000

# Length of investment period (in months)
investPeriod = 6

# divide the factor returns by  100
factorRet = factorRet/100

#rf and factor returns
riskFree = factorRet['RF']
factorRet = factorRet.loc[:,factorRet.columns != 'RF'];

#Identify the tickers and the dates
tickers = adjClose.columns
dates   = factorRet.index

# Calculate the stocks monthly excess returns
# pct change and drop the first null observation
returns = adjClose.pct_change(1).iloc[1:, :]
returns = returns  - np.diag(riskFree.values) @ np.ones_like(returns.values)
# Align the price table to the asset and factor returns tables by discarding the first observation.
adjClose = adjClose.iloc[1:,:]

assert adjClose.index[0] == returns.index[0]
assert adjClose.index[0] == factorRet.index[0]


# 3. Results

In [8]:
def run_strategy(num_periods,lambda_RP, lamda_S, z, model, cal, file_1, file_2):

    adjClose = pd.read_csv(file_1, index_col=0)
    factorRet = pd.read_csv(file_2, index_col=0)

    adjClose.index = pd.to_datetime(adjClose.index)
    factorRet.index = pd.to_datetime(factorRet.index)

    # Initial budget to invest ($100,000)
    initialVal = 100000

    # Length of investment period (in months)
    investPeriod = 6

    # divide the factor returns by  100
    factorRet = factorRet/100

    #rf and factor returns
    riskFree = factorRet['RF']
    factorRet = factorRet.loc[:,factorRet.columns != 'RF'];

    #Identify the tickers and the dates
    tickers = adjClose.columns
    dates   = factorRet.index

    # Calculate the stocks monthly excess returns
    # pct change and drop the first null observation
    returns = adjClose.pct_change(1).iloc[1:, :]
    returns = returns  - np.diag(riskFree.values) @ np.ones_like(returns.values)
    # Align the price table to the asset and factor returns tables by discarding the first observation.
    adjClose = adjClose.iloc[1:,:]

    assert adjClose.index[0] == returns.index[0]
    assert adjClose.index[0] == factorRet.index[0]



    testStart = returns.index[0] + pd.offsets.DateOffset(years=cal)
    investPeriod = 5
    testEnd = testStart + pd.offsets.DateOffset(months=investPeriod) - pd.offsets.DateOffset(days=1)
    calEnd = testStart - pd.offsets.DateOffset(days=1)

    NoPeriods = math.ceil((returns.index[-1].to_period('M') - testStart.to_period('M')).n / investPeriod)
    n = len(tickers)

    x = np.zeros([n, NoPeriods])
    x0 = np.zeros([n, NoPeriods])
    currentVal = np.zeros([NoPeriods, 1])
    turnover = np.zeros([NoPeriods, 1])
    toDay = 0
    portfValue = []

    for t in range(NoPeriods):
        periodReturns = returns[returns.index <= calEnd]
        periodFactRet = factorRet[factorRet.index <= calEnd]

        current_price_idx = (calEnd - pd.offsets.DateOffset(months=1) <= adjClose.index)&(adjClose.index <= calEnd)
        currentPrices = adjClose[current_price_idx]

        periodPrices_idx = (testStart <= adjClose.index)&(adjClose.index <= testEnd)
        periodPrices = adjClose[periodPrices_idx]

        if len(periodPrices) != investPeriod or len(currentPrices) != 1:
            break  # Skip this period if data is missing

        if t == 0:
            currentVal[0] = initialVal
        else:
            currentVal[t] = currentPrices @ NoShares.values.T
            x0[:,t] = currentPrices.values * NoShares.values / currentVal[t]

        x[:,t] = grid_search_function(periodReturns, periodFactRet, num_periods,lambda_RP, lamda_S, z, model)

        if t > 0:
            turnover[t] = np.sum(np.abs(x[:,t] - x0[:,t]))

        NoShares = x[:,t] * currentVal[t] / currentPrices

        fromDay = toDay
        toDay = toDay + len(periodPrices)

        portfValue.append(periodPrices @ NoShares.values.T)

        testStart = testStart + pd.offsets.DateOffset(months=investPeriod)
        testEnd = testStart + pd.offsets.DateOffset(months=investPeriod) - pd.offsets.DateOffset(days=1)
        calEnd = testStart - pd.offsets.DateOffset(days=1)

    if len(portfValue) == 0:
        return -np.inf, np.nan  # Invalid run

    portfValue = pd.concat(portfValue, axis=0)
    portfRets = portfValue.pct_change(1).iloc[1:,:]
    portfExRets = portfRets.subtract(riskFree[(riskFree.index >= portfRets.index[0]) & (riskFree.index <= portfRets.index[-1])], axis=0)

    SR = ((portfExRets + 1).apply(gmean, axis=0) - 1) / portfExRets.std()
    avgTurnover = np.mean(turnover[1:])

    return SR[0], avgTurnover


In [ ]:
for model in range(3):
    best_ratio = -np.inf
    best_params = {}

    # Model 0 (Risk Parity)
    if model == 0:
        lambda_RP = 0.75  # example value
        lamda_S = "N/A"
        z = "N/A"
        for num_periods in [30,36]:
            SRs = []
            turnovers = []
            for file in [["MIE377_AssetPrices_1.csv", "MIE377_FactorReturns_1.csv"], 
                         ["MIE377_AssetPrices_2.csv", "MIE377_FactorReturns_2.csv"]]:
                for cal in [3, 5, 8]:
                    print(f"\nTesting model {model}, lambda_RP={lambda_RP}, lambda_S={lamda_S}, z={z}, num_periods={num_periods}, cal={cal}")
                    SR, turnover = run_strategy(num_periods=num_periods, lambda_RP=lambda_RP, lamda_S=None, z=None,
                                                model=model, cal=cal, file_1=file[0], file_2=file[1])
                    print(f"--> SR = {SR:.4f}, Turnover = {turnover:.4f}")
                    SRs.append(SR)
                    turnovers.append(turnover)

            avg_SR = np.mean(SRs)
            avg_turnover = np.mean(turnovers)
            avg_ratio = avg_SR / avg_turnover

            if avg_ratio > best_ratio:
                best_ratio = avg_ratio
                best_params = {
                    'model': model,
                    'lambda_RP': lambda_RP,
                    'lambda_S': lamda_S,
                    'z': z,
                    'num_periods': num_periods,
                    'SR': avg_SR,
                    'turnover': avg_turnover
                }

    # Model 1 (Sharpe)
    elif model == 1:
        lambda_RP = "N/A"
        z = "N/A"
        for lamda_S in [8, 12, 16]:
            for num_periods in [30, 36]:
                SRs = []
                turnovers = []
                for file in [["MIE377_AssetPrices_1.csv", "MIE377_FactorReturns_1.csv"],
                             ["MIE377_AssetPrices_2.csv", "MIE377_FactorReturns_2.csv"]]:
                    for cal in [3, 5, 8]:
                        print(f"\nTesting model {model}, lambda_RP={lambda_RP}, lambda_S={lamda_S}, z={z}, num_periods={num_periods}, cal={cal}")
                        SR, turnover = run_strategy(num_periods=num_periods, lambda_RP=None, lamda_S=lamda_S, z=None,
                                                    model=model, cal=cal, file_1=file[0], file_2=file[1])
                        print(f"--> SR = {SR:.4f}, Turnover = {turnover:.4f}")
                        SRs.append(SR)
                        turnovers.append(turnover)

                avg_SR = np.mean(SRs)
                avg_turnover = np.mean(turnovers)
                avg_ratio = avg_SR / avg_turnover

                if avg_ratio > best_ratio:
                    best_ratio = avg_ratio
                    best_params = {
                        'model': model,
                        'lambda_RP': lambda_RP,
                        'lambda_S': lamda_S,
                        'z': z,
                        'num_periods': num_periods,
                        'SR': avg_SR,
                        'turnover': avg_turnover
                    }

    # Model 2 (Mixed)
    elif model == 2:
        for lambda_RP in [0.25, 0.5, 0.75]:
            for lamda_S in [8, 12, 16]:
                for z in [0.25, 0.5, 0.75]:
                    for num_periods in [30, 36]:
                        SRs = []
                        turnovers = []
                        for file in [["MIE377_AssetPrices_1.csv", "MIE377_FactorReturns_1.csv"],
                                     ["MIE377_AssetPrices_2.csv", "MIE377_FactorReturns_2.csv"]]:
                            for cal in [3]:
                                print(f"\nTesting model {model}, lambda_RP={lambda_RP}, lambda_S={lamda_S}, z={z}, num_periods={num_periods}, cal={cal}")
                                SR, turnover = run_strategy(num_periods=num_periods, lambda_RP=lambda_RP,
                                                            lamda_S=lamda_S, z=z, model=model, cal=cal,
                                                            file_1=file[0], file_2=file[1])
                                print(f"--> SR = {SR:.4f}, Turnover = {turnover:.4f}")
                                SRs.append(SR)
                                turnovers.append(turnover)

                        avg_SR = np.mean(SRs)
                        avg_turnover = np.mean(turnovers)
                        avg_ratio = avg_SR / avg_turnover

                        if avg_ratio > best_ratio:
                            best_ratio = avg_ratio
                            best_params = {
                                'model': model,
                                'lambda_RP': lambda_RP,
                                'lambda_S': lamda_S,
                                'z': z,
                                'num_periods': num_periods,
                                'SR': avg_SR,
                                'turnover': avg_turnover
                            }



Testing model 0, lambda_RP=0.75, lambda_S=N/A, z=N/A, num_periods=30, cal=3
--> SR = 0.2105, Turnover = 0.1099

Testing model 1, lambda_RP=N/A, lambda_S=8, z=N/A, num_periods=30, cal=3
--> SR = 0.2398, Turnover = 0.3736

Testing model 1, lambda_RP=N/A, lambda_S=8, z=N/A, num_periods=30, cal=5
--> SR = 0.1797, Turnover = 0.3896

Testing model 1, lambda_RP=N/A, lambda_S=8, z=N/A, num_periods=30, cal=8


KeyboardInterrupt: 

In [ ]:
# Prepare data for a nice table
table_data = []
for model, result in best_results.items():
    table_data.append({
        'Model': model,
        'lambda_RP': result.get('lambda_RP', 'N/A'),
        'lambda_S': result.get('lambda_S', 'N/A'),
        'z': result.get('z', 'N/A'),
        'num_periods': result['num_periods'],
        'SR': result['SR'],
        'Turnover': result['turnover']
    })

# Convert to DataFrame for better readability
df_results = pd.DataFrame(table_data)

# Display the results in a table format
print("\nBest Results Summary:")
print(df_results)



Best Results Summary:
   Model lambda_RP lambda_S     z  num_periods        SR  Turnover
0      0      0.75      N/A   N/A           30  0.210506  0.109856
1      1       N/A     0.75   N/A           30  0.216605  0.268931
2      2      0.75        8  0.25           30  0.203805  0.142854
